In [1]:
#NOTES
#normalize images by deviding by 255

In [1]:
from scipy import misc
from sklearn.utils import shuffle
import numpy as np
def mass_resize(images, size=(128,128)):
    arr = np.zeros((len(images),size[0],size[1]))
    for i,im in enumerate(images):
        arr[i] = misc.imresize(im,size)
    return arr


In [2]:
import tensorflow as tf
mnist =  tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

size = (128,128)

x_train = mass_resize(x_train,size)
x_test = mass_resize(x_test,size)

c:\users\ablac\miniconda3\envs\expressions\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  import sys


In [3]:
from matplotlib.pyplot import imshow
X,Y = x_train,y_train
X,Y = shuffle(X,Y)
imshow(X[0])

print(Y[0])

5


In [4]:
num_classes = 10
x = tf.placeholder(dtype=tf.float32, shape=(None,size[0],size[1]),name='x')
y = tf.placeholder(dtype=tf.uint8, shape=(None),name="y")
lr = tf.placeholder(dtype=tf.float32, shape=[], name="lr")

In [5]:
one_hot = tf.one_hot(y,num_classes)
reshaped = tf.reshape(x,[-1,size[0],size[1],1])

import tensorflow.contrib.slim as slim

In [7]:
#batch,128,128,1

#first convolution
conv = tf.contrib.layers.conv2d(
    reshaped,
    32,
    (9,9),
    stride=(1,1),
    padding='VALID',
    weights_initializer=tf.initializers.random_normal)
#batch,120,120,32

#subsample
pool = tf.contrib.layers.max_pool2d(
    conv,
    (4,4),
    stride=(4,4))
#batch,30,30,32

#second convolution
conv2 = tf.contrib.layers.conv2d(
    pool,
    64,
   (11,11),
    stride=(1,1),
    padding='VALID',
    weights_initializer=tf.initializers.random_normal)
#batch,20,20,64

#subsample
pool2 = tf.contrib.layers.max_pool2d(
    conv2,
    (4,4),
    stride=(4,4))
#batch,5,5,64

#flatten
flat = tf.contrib.layers.flatten(pool2)
#batch,1600

#fully connected
fc = tf.contrib.layers.fully_connected(
    flat,
    1056,
    activation_fn=tf.nn.relu,
    weights_initializer=tf.initializers.random_normal)
#batch,256

#logits = slim.fully_connected(flat, 10, activation_fn=None, normalizer_fn=None)
logits = tf.contrib.layers.fully_connected(
    fc,
    num_classes,
    activation_fn=None)

prediction = tf.nn.softmax(logits,)

In [8]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=one_hot)
loss_op = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(lr).minimize(loss_op)
correct_pred = tf.equal(tf.argmax(prediction,1), tf.argmax(one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [9]:

epochs = 40
learn_rate = .0001
batch_size = 256

with tf.Session() as sess:
    tx,ty = shuffle(x_test,y_test)
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch in range(epochs):
        epoch_loss = 0
        batch_i = 1
        #for X,Y in next_batch(x_train,y_train,batch_size):
        X,Y = x_train,y_train
        X,Y = shuffle(X,Y)
        
        for i in range(0,len(X),batch_size):
            x_batch = X[i:i+batch_size]
            y_batch = Y[i:i+batch_size]
            _,c = sess.run([optimizer,loss_op],feed_dict={x:x_batch,y:y_batch,lr:learn_rate})
            epoch_loss += c
            if batch_i % 100 == 0:
                valacc = sess.run(accuracy,feed_dict={x:x_batch,y:y_batch})
                loss, trainacc = sess.run([loss_op, accuracy], feed_dict={x:x_batch,y:y_batch})
                print("epoch {} \tbatch: {}\ttrain accuracy: {}\tloss: {}\tval acc: {}"
                      .format(epoch,batch_i,round(float(trainacc),4),round(float(loss),4), round(float(valacc),4)))               
                
            batch_i += 1
        print('Epoch', epoch, 'completed out of',epochs,'loss:',epoch_loss)
            
    
        
        print('Test Accuracy:',accuracy.eval({x:tx[:100], y:ty[:100]}))

            

epoch 0 	batch: 100	train accuracy: 0.582	loss: 259204.375	val acc: 0.582
epoch 0 	batch: 200	train accuracy: 0.7578	loss: 119277.6172	val acc: 0.7578
Epoch 0 completed out of 40 loss: 103922787.9140625
Test Accuracy: 0.77
epoch 1 	batch: 100	train accuracy: 0.8516	loss: 75840.9453	val acc: 0.8516
epoch 1 	batch: 200	train accuracy: 0.8555	loss: 56840.7617	val acc: 0.8555
Epoch 1 completed out of 40 loss: 16283128.296875
Test Accuracy: 0.87
epoch 2 	batch: 100	train accuracy: 0.9141	loss: 32574.7969	val acc: 0.9141
epoch 2 	batch: 200	train accuracy: 0.9141	loss: 33002.3906	val acc: 0.9141
Epoch 2 completed out of 40 loss: 10292966.097167969
Test Accuracy: 0.91
epoch 3 	batch: 100	train accuracy: 0.9219	loss: 29874.1016	val acc: 0.9219
epoch 3 	batch: 200	train accuracy: 0.9336	loss: 36635.1172	val acc: 0.9336
Epoch 3 completed out of 40 loss: 7654404.576171875
Test Accuracy: 0.9
epoch 4 	batch: 100	train accuracy: 0.8906	loss: 62628.9922	val acc: 0.8906
epoch 4 	batch: 200	train accur

epoch 37 	batch: 100	train accuracy: 1.0	loss: 0.0	val acc: 1.0
epoch 37 	batch: 200	train accuracy: 0.9883	loss: 461.4282	val acc: 0.9883
Epoch 37 completed out of 40 loss: 193967.28393554688
Test Accuracy: 0.96
epoch 38 	batch: 100	train accuracy: 1.0	loss: 0.0	val acc: 1.0
epoch 38 	batch: 200	train accuracy: 0.9961	loss: 873.7026	val acc: 0.9961
Epoch 38 completed out of 40 loss: 238248.2107849121
Test Accuracy: 0.96
epoch 39 	batch: 100	train accuracy: 0.9961	loss: 124.9678	val acc: 0.9961
epoch 39 	batch: 200	train accuracy: 0.9961	loss: 463.9555	val acc: 0.9961
Epoch 39 completed out of 40 loss: 151627.134973526
Test Accuracy: 0.96


In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes])),}


    l1 = tf.add(tf.matmul(data,hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2,hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.matmul(l3,output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        
        for epoch in range(epochs):
            batch_i = 1
            epoch_loss = 0
            #for X,Y in next_batch(x_train,y_train,batch_size):
            X,Y = x_train,y_train
            X,Y = shuffle(X,Y)

            for i in range(0,len(X),batch_size):
                x_batch = X[i:i+batch_size]
                y_batch = Y[i:i+batch_size]
                sess.run(optimizer,feed_dict={x:x_batch,y:y_batch,lr:learn_rate})
                if batch_i % 10 == 0:
                    valacc = sess.run(accuracy,feed_dict={x:x_test[:100],y:y_test[:100]})
                    loss, trainacc = sess.run([loss_op, accuracy], feed_dict={x:x_batch,y:y_batch})
                    print("epoch {} \tbatch: {}\ttrain accuracy: {}\tloss: {}\tval acc: {}"
                          .format(epoch,batch_i,round(float(trainacc),4),round(float(loss),4), round(float(valacc),4)))               

                batch_i += 1

train_neural_network(x)